In [1]:
! pip install predictionguard langchain langchain_community google-search-results

In [2]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import PredictionGuard
import os
from getpass import getpass
from langchain.tools import Tool

In [3]:
# pg_access_token = getpass('Enter your Prediction Guard access token: ')
pg_access_token = 'u7ojcxj2jrQ9ygro7sRCXETHfgQf84NmhnXUFEsk'
os.environ['PREDICTIONGUARD_TOKEN'] = pg_access_token

In [13]:
# Initialize PredictionGuard LLM
llm = PredictionGuard(model="Hermes-3-Llama-3.1-70B", predictionguard_api_key=pg_access_token)

# Define the function to extract meeting information
def extract_meeting_info(transcript: str):
    prompt = f"""
    Extract key information from the following meeting transcript:
    
    1. **Meeting Date and Time**
    2. **Participants**
    3. **Agenda Items Discussed**
    4. **Action Items** (who is responsible and deadlines)
    5. **Decisions Made**
    
    Transcript:
    {transcript}

    Format the output clearly.
    """
    return llm.invoke(prompt)

# Wrap this function as a LangChain Tool
meeting_info_tool = Tool(
    name="ExtractMeetingInfo",
    func=extract_meeting_info,
    description="Extracts structured meeting information from a transcript."
)

In [14]:
# Initialize the agent with tools and LLM
agent = initialize_agent(
    tools=[meeting_info_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [15]:
transcript = """
Alice Johnson: Good morning, everyone. Let's review our Q1 progress. 
Bob Smith: We've completed 80 percent of development, but API integration is delayed.
Carol Davis: Marketing is ready but needs a final release date.
David Lee: We can allocate $10,000 for marketing but need to adjust the budget.
Alice Johnson: Let's schedule a call on 13th February at 2 PM.
"""

response = agent.run(transcript)
print(response)



> Entering new AgentExecutor chain...
 To extract the key information from this meeting transcript, I will use the ExtractMeetingInfo tool.

Action: ExtractMeetingInfo
Action Input: Alice Johnson: Good morning, everyone. Let's review our Q1 progress. 
Bob Smith: We've completed 80 percent of development, but API integration is delayed.
Carol Davis: Marketing is ready but needs a final release date. 
David Lee: We can allocate $10,000 for marketing but need to adjust the budget.
Alice Johnson: Let's schedule a call on 13th February at 2 PM.

Observation: 



**Meeting Date and Time:** 13th February at 2 PM

**Participants:**
- Alice Johnson
- Bob Smith
- Carol Davis
- David Lee

**Agenda Items Discussed:**
- Q1 progress review
- Development progress (80% complete)
- API integration delay
- Marketing readiness and requirement of a final release date
- Budget allocation for marketing ($10,000)

**Action Items:**
- Schedule a call on 13th February at 2 PM (Alice Johnson)

**Decisions Mad

In [16]:
transcript = """
Hey there. I am interested in using PG.
"""

response = agent.run(transcript)
print(response)



> Entering new AgentExecutor chain...
 To provide a helpful response, I need to understand what "PG" refers to in this context. It could mean a few different things.

Action: ExtractMeetingInfo
Action Input: Hey there. I am interested in using PG.
Observation: 




**Meeting Date and Time:** Not specified

**Participants:** Not specified

**Agenda Items Discussed:** 
- Interest in using PG

**Action Items:** 
- None specified

**Decisions Made:**
- None specified    



The provided text appears to be a brief message expressing interest in using PG, but it does not contain enough information to extract a meeting date/time, participants, agenda items, action items or decisions made. It seems to be a casual message rather than a formal meeting transcript.
Thought: Based on the limited information provided, I don't have enough context to determine what "PG" refers to. It could be an acronym for a product, service, or organization the person is interested in using. To provide a more help